<a href="https://colab.research.google.com/github/anilkaushikwireless/email_wrapper/blob/master/my_graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from math import pi
import pandas as pd

from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.transform import cumsum


from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure


from bokeh.models.widgets import DataTable, TableColumn, HTMLTemplateFormatter
from bokeh.io import show

import matplotlib.pyplot as plt
import numpy as np
import itertools

from sklearn.metrics import confusion_matrix

output_notebook()



def draw_vertical_bar(x_axis_data, y_axis_data, title):
  
  source = ColumnDataSource(data=dict(x_axis_data=x_axis_data, y_axis_data=y_axis_data, color=Spectral6[:4]))

  p1 = figure(x_range=x_axis_data, plot_height=250, y_range=(0, 9), title=title)
  p1.vbar(x='x_axis_data', top='y_axis_data', width=0.5, color='color', legend_field="x_axis_data", source=source)

  p1.xgrid.grid_line_color = None
  p1.legend.orientation = "vertical"
  p1.legend.location = "top_center"

  return p1


def draw_horizontal_bar(x_axis_data, y_axis_data, title):
  score = x_axis_data
  data = y_axis_data

  p = figure(
  y_range=score,
  title = title,
  x_axis_label =title,
  plot_width=800,
  plot_height=600,
  tools=""
  )

  p.hbar(
    y=score,
    right=data,
    left=0,
    height=0.4,
    color=Spectral6[:len(x_axis_data)-1],
    fill_alpha=0.5
  )

  return p

def ridge(category, data, scale=20):
    return list(zip([category]*len(data), scale*data))



#Pie chart
def draw_pie_chart():
  x = { 'Column': 157, 'Non txt column': 93, 'Text column': 89 }

  data = pd.Series(x).reset_index(name='value').rename(columns={'index':'meta'})
  data['angle'] = data['value']/data['value'].sum() * 2*pi
  data['color'] = ["red", "green", "blue"]

  p3 = figure(plot_height=350, title="Meta data", toolbar_location=None,
          tools="hover", tooltips="@meta: @value")

  p3.wedge(x=0, y=1, radius=0.4,
          start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
          line_color="white", fill_color='color', legend_field='meta', source=data)
  
  return p3


def draw_confusion_matrix(conf_mat,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
  
    from bokeh.palettes import Category10_10 as palette, viridis
    from bokeh.models import HoverTool
    from collections import OrderedDict

    actual = np.array([['1', '1'], ['0', '0']])
    actual = list(actual.reshape(4))

    predicted = np.array([['1', '0'], ['1', '0']])
    predicted = list(predicted.reshape(4))

    count = list(conf_mat.reshape(4))

    alpha = [count_val / conf_mat.sum() + 0.2 for count_val in count]

    source = ColumnDataSource(
        data=dict(predicted=predicted, actual=actual, count=count, alpha=alpha)
    )
    p = figure(title=title, tools="hover,save",
                          y_range=['0', '1'], x_range=['1', '0'],
                          x_axis_label='predicted label', 
                          y_axis_label='true label',
                          x_axis_location='above',
                          width=200, height=200)

    rectwidth = 0.9
    p.rect('predicted', 'actual', rectwidth, rectwidth, source=source,
          color=palette[0], alpha='alpha',line_width=1)

    hover = p.select(dict(type=HoverTool))
    hover.tooltips = OrderedDict([
        ('predicted', '@predicted'),
        ('actual', '@actual'),
        ('count', '@count'),
    ])
    return p

#tablular format for all teh metrices
def draw_all_metrices():
  dict1 = {'x':[0]*6,'y':[0,1,0,1,0,1]}
  source = ColumnDataSource(data=dict1)

  template="""
  <div style="background:<%= 
      (function colorfromint(){
          if(value == 1){
              return("blue")}
          else{return("red")}
          }()) %>; 
      color: white"> 
  <%= value %></div>
  """

  formater =  HTMLTemplateFormatter(template=template)
  columns = [
      TableColumn(field="Threshold", title="Threshold"),
      TableColumn(field="Accuracy", title="Accuracy"),
      TableColumn(field="Precision", title="Precision"),
      TableColumn(field="Recall", title="Recall"),
      TableColumn(field="F1 score", title="F1 score",formatter=formater)
  ]

  data_table = DataTable(source=source, columns=columns, width=200)
  return data_table


x = list(range(11))
y0 = x
y1 = [10 - i for i in x]
y2 = [abs(i - 5) for i in x]

# create three plots

data = ['Unseen data(before)', 'Unseen data(after)', 'Training data(before)', 'Training data(after)']
fi_score = [91, 92, 91, 93 ]  
auc_score = [64, 75, 62, 72 ]  
roc_score = [5, 3, 4, 2 ]  
precision_score = [5, 3, 4, 2 ]  
recall_score = [5, 3, 4, 2 ]  
accuracy_score = [5, 3, 4, 2 ]  


y_true = ["cat", "ant", "cat", "cat"]
y_pred = ["ant", "ant", "cat", "cat"]
cmatrix = confusion_matrix(y_true, y_pred, labels=["ant","cat"])

p1 = draw_horizontal_bar(data, accuracy_score, "Accuracy(%)");
p2 = draw_horizontal_bar(data,fi_score,"F1 Score(%)");
p3 = draw_horizontal_bar(data, precision_score, "Precision(%)");
p4 = draw_horizontal_bar(data, recall_score, "Recall(%)");


# make a grid
grid = gridplot([[p1, p2], [p3, p4]], plot_width=400, plot_height=400)
show(grid)
p5 = draw_horizontal_bar(data, roc_score, "ROC(%)");
p6 = draw_horizontal_bar(data, auc_score, "AUC(%)");

cm1 = draw_confusion_matrix(cmatrix, 
                      normalize    = False,
                      target_names = ['1', '0'],
                      title        = "Confusion Matrix(Before)")

cm2 = draw_confusion_matrix(cmatrix, 
                      normalize    = False,
                      target_names = ['1', '0'],
                      title        = "Confusion Matrix(After)")

grid = gridplot([[p5, p6], [cm1, cm2]], plot_width=400, plot_height=400)
show(grid)


######## META DATA ##############################################
labels = ['duplicate_rows', 'null_values', 'count_Tokens', 'count_punctuations'
          ,'count_hashtags', 'count_nnumeric', 'count_special_symbols', 'count_emojis'
          ,'count_extra_spaces', 'count_stopwords', 'vocab_size']
duplicate_rows = 692
null_values = 0
count_Tokens = 41770 
count_punctuations = 13
count_hashtags = 57
count_nnumeric = 11593
count_tags = 23631
count_special_symbols = 95
count_emojis = 33
count_extra_spaces = 11770
count_stopwords = 140
vocab_size = 26476

before_cleaning_val = [duplicate_rows, null_values, count_Tokens, count_punctuations, count_hashtags
                        , count_nnumeric, count_special_symbols, count_emojis, count_extra_spaces
                        , count_stopwords, vocab_size]

p7 = draw_horizontal_bar(labels, before_cleaning_val, "Meta data before cleaning");
p8 = draw_horizontal_bar(labels, before_cleaning_val, "Meta data before cleaning");
p9 = draw_horizontal_bar(labels, before_cleaning_val, "Meta data before cleaning");
p10 = draw_horizontal_bar(labels, before_cleaning_val, "Meta data before cleaning");
grid = gridplot([[p7, p8], [p9, p10]], plot_width=400, plot_height=400)
show(grid)

